# Clusterização

[**Premier League:** Premier League Match Data as well as Financial Data of all clubs in the 18-19](https://www.kaggle.com/thesiff/premierleague1819)


Execute  os  algoritmosde clusterizacao citados a seguir e compare os resultados. 
- Execute os algoritmos de agrupamento K-means e Hierarquico com os seguintes valores de K:2, 5, 10 e 100. Compare os agrupamento resultantes dos 2 algoritmos. 
- Escolha  um  numero  fixo  de  K  e  altere  o  parametro  do  K-Means  referente  ao  numeromaximo  de  iteracoes:  1,  10  e  100  e  o  parametro  de  Linkage  do  Hierarquico,  quaisdiferencas puderam ser observadas? 
- Faca uma comparacao entre os 2 algoritmos, qual voce acha que teve o melhordesempenho e por que? 

**Bonus**:  normalizar  os  dados  e  executar  novamente  os  algoritmos  para  analisar  os  novos resultados gerados. Em seguida, fazer os graficos dos agrupamentos dos dados brutos vs dados normalizados, e um boxplot para mostrar a dispersao destes dados

In [1]:
import pandas as pd
import sklearn as sk
from sklearn import model_selection
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import seaborn as sns


from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans

In [2]:
epl_df = pd.read_csv("epl_1819.csv")
transfer_df = pd.read_csv("transfer.csv")

In [3]:
epl_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 44 columns):
Team                             20 non-null object
category                         20 non-null object
general_league_position          20 non-null int64
finance _live_games_televised    20 non-null int64
finance _tv_revenue              20 non-null int64
general_matches_played           20 non-null int64
general_won                      20 non-null int64
general_draw                     20 non-null int64
general_lost                     20 non-null int64
attack_scored                    20 non-null int64
defence_goals_conceeded          20 non-null int64
general_goal_difference          20 non-null int64
general_points                   20 non-null int64
general_squad_size               20 non-null int64
general_squad_average_age        20 non-null float64
general_squad_foreigners         20 non-null int64
finance _team_market             20 non-null int64
finance _market_average   

In [4]:
transfer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 12 columns):
team        20 non-null object
end_2019    20 non-null float64
end_2018    20 non-null float64
end_2017    20 non-null float64
end_2016    20 non-null float64
end_2015    20 non-null float64
end_2014    20 non-null float64
end_2013    20 non-null float64
end_2012    20 non-null float64
end_2011    20 non-null float64
end_2010    20 non-null float64
Total       20 non-null float64
dtypes: float64(11), object(1)
memory usage: 2.0+ KB


In [5]:
epl_df.head()

,Team,category,general_league_position,finance _live_games_televised,finance _tv_revenue,general_matches_played,general_won,general_draw,general_lost,attack_scored,...,defence_saves,defence_blocks,defence_interceptions,defence_tackles,defence_tackles_last_man,defence_clearances,defence_clearances_headed,defence_penalty_conceeded,attack_posession,attack_pass_accuracy
0,Manchester City,Champions League,1,26,150986355,38,32,2,4,95,...,58,63,362,518,0,543,295,4,64.0,89.0
1,Liverpool,Champions League,2,29,152425146,38,30,7,1,89,...,76,78,314,610,2,639,317,1,58.8,84.4
2,Chelsea,Champions League Qualification,3,25,146030216,38,21,9,8,63,...,85,73,351,618,2,593,330,2,59.9,87.6
3,Tottenham,Champions League Qualification,4,26,145230801,38,23,2,13,67,...,121,112,326,626,3,770,411,5,56.7,83.2
4,Arsenal,Europa League,5,25,142193180,38,21,7,10,73,...,133,118,412,609,2,762,404,7,56.1,83.3


In [6]:
transfer_df.head()

,team,end_2019,end_2018,end_2017,end_2016,end_2015,end_2014,end_2013,end_2012,end_2011,end_2010,Total
0,Manchester City,-22.99,-226.15,-178.15,-140.86,-57.68,-104.70,-17.65,-59.85,-146.46,-116.35,-1070.83
1,Manchester United,-52.15,-152.90,-137.85,-53.93,-146.06,-75.33,-65.57,-47.84,-11.99,77.07,-666.55
2,Tottenham,5.35,-17.70,-31.30,16.58,-4.32,13.55,-0.47,33.80,-23.67,-8.60,-16.78
3,Liverpool,-143.70,20.62,5.48,-34.85,-52.16,-25.50,-60.15,-43.63,3.98,5.05,-324.86
4,Chelsea,-148.25,-60.00,-24.40,-3.01,7.15,-52.92,-84.25,-64.05,-105.00,-26.20,-560.93


### Retirando a virgula de algumas colunas que foram reconhecidas como objeto

In [7]:
columns = ["attack_passes", "attack_passes_long", "attack_passes_back", "defence_clearances"]
for column in columns:
    epl_df[column] = epl_df[column].str.replace(",", "").astype(int)

### Ambas as tabelas tem o nome do Time, precisamos tratar isso antes de unir para não duplicar dados

- o .strip() retira os espaços no começo e fim da string
- na tabela transfer_df o header Team está com letra minuscula

In [8]:
transfer_df.rename(columns={'team':'Team'}, inplace=True)


In [9]:
transfer_df["Team"] = transfer_df["Team"].str.strip()
transfer_df["Team"]

0       Manchester City
1     Manchester United
2             Tottenham
3             Liverpool
4               Chelsea
5               Arsenal
6               Burnley
7               Everton
8             Leicester
9      Newcastle United
10       Crystal Palace
11          Bournemouth
12             West Ham
13              Watford
14             Brighton
15         Huddersfield
16          Southampton
17        Wolverhampton
18              Cardiff
19               Fulham
Name: Team, dtype: object

In [10]:
epl_df["Team"] = epl_df.sort_values("Team")["Team"].str.strip()
epl_df

,Team,category,general_league_position,finance _live_games_televised,finance _tv_revenue,general_matches_played,general_won,general_draw,general_lost,attack_scored,...,defence_saves,defence_blocks,defence_interceptions,defence_tackles,defence_tackles_last_man,defence_clearances,defence_clearances_headed,defence_penalty_conceeded,attack_posession,attack_pass_accuracy
0,Manchester City,Champions League,1,26,150986355,38,32,2,4,95,...,58,63,362,518,0,543,295,4,64.0,89.0
1,Liverpool,Champions League,2,29,152425146,38,30,7,1,89,...,76,78,314,610,2,639,317,1,58.8,84.4
2,Chelsea,Champions League Qualification,3,25,146030216,38,21,9,8,63,...,85,73,351,618,2,593,330,2,59.9,87.6
3,Tottenham,Champions League Qualification,4,26,145230801,38,23,2,13,67,...,121,112,326,626,3,770,411,5,56.7,83.2
4,Arsenal,Europa League,5,25,142193180,38,21,7,10,73,...,133,118,412,609,2,762,404,7,56.1,83.3
5,Manchester United,Europa League,6,27,142512868,38,19,9,10,65,...,122,128,355,581,2,785,370,6,53.3,82.3
6,Wolverhampton,Europa League Qualification,7,15,127165114,38,16,9,13,47,...,100,148,501,720,2,824,464,4,47.4,78.2
7,Everton,No UEFA Competitions,8,18,128603905,38,15,9,14,54,...,94,85,456,700,4,825,430,5,51.0,77.3
8,Leicester,No UEFA Competitions,9,15,123328078,38,15,7,16,51,...,90,96,436,676,4,945,500,5,50.8,79.2
9,West Ham,No UEFA Competitions,10,16,122528663,38,15,7,16,52,...,148,141,457,723,5,855,397,5,49.0,78.7


### Unindo as duas tabelas

In [11]:
result = pd.merge(epl_df, transfer_df, on="Team")
result

,Team,category,general_league_position,finance _live_games_televised,finance _tv_revenue,general_matches_played,general_won,general_draw,general_lost,attack_scored,...,end_2018,end_2017,end_2016,end_2015,end_2014,end_2013,end_2012,end_2011,end_2010,Total
0,Manchester City,Champions League,1,26,150986355,38,32,2,4,95,...,-226.15,-178.15,-140.86,-57.68,-104.70,-17.65,-59.85,-146.46,-116.35,-1070.83
1,Liverpool,Champions League,2,29,152425146,38,30,7,1,89,...,20.62,5.48,-34.85,-52.16,-25.50,-60.15,-43.63,3.98,5.05,-324.86
2,Chelsea,Champions League Qualification,3,25,146030216,38,21,9,8,63,...,-60.00,-24.40,-3.01,7.15,-52.92,-84.25,-64.05,-105.00,-26.20,-560.93
3,Tottenham,Champions League Qualification,4,26,145230801,38,23,2,13,67,...,-17.70,-31.30,16.58,-4.32,13.55,-0.47,33.80,-23.67,-8.60,-16.78
4,Arsenal,Europa League,5,25,142193180,38,21,7,10,73,...,3.35,-102.65,-24.00,-91.18,-37.10,9.85,12.82,-13.40,35.70,-281.72
5,Manchester United,Europa League,6,27,142512868,38,19,9,10,65,...,-152.90,-137.85,-53.93,-146.06,-75.33,-65.57,-47.84,-11.99,77.07,-666.55
6,Wolverhampton,Europa League Qualification,7,15,127165114,38,16,9,13,47,...,-18.29,-33.11,5.73,-2.02,4.26,15.15,-9.45,-13.78,-18.58,-159.30
7,Everton,No UEFA Competitions,8,18,128603905,38,15,9,14,54,...,-76.72,-25.10,-37.90,-38.26,14.30,-2.90,19.60,4.90,5.04,-208.19
8,Leicester,No UEFA Competitions,9,15,123328078,38,15,7,16,51,...,-39.95,-25.55,-40.45,-22.86,0.65,-1.72,-16.71,-2.50,-0.81,-168.69
9,West Ham,No UEFA Competitions,10,16,122528663,38,15,7,16,52,...,12.22,-42.50,-34.19,-30.75,-23.47,-18.85,-2.49,-10.88,19.65,-221.72


### Tratando a categoria das ligas, que é objeto

In [12]:
category_encoding = pd.get_dummies(result["category"])
category_encoding

,Champions League,Champions League Qualification,Europa League,Europa League Qualification,No UEFA Competitions,Relegated
0,1,0,0,0,0,0
1,1,0,0,0,0,0
2,0,1,0,0,0,0
3,0,1,0,0,0,0
4,0,0,1,0,0,0
5,0,0,1,0,0,0
6,0,0,0,1,0,0
7,0,0,0,0,1,0
8,0,0,0,0,1,0
9,0,0,0,0,1,0


### Unindo os dados do Encoding

In [13]:
result_encoding = pd.concat([result, category_encoding], axis=1)

result_encoding = result_encoding.drop(["category", "Team"], axis = 1)
result_encoding

,general_league_position,finance _live_games_televised,finance _tv_revenue,general_matches_played,general_won,general_draw,general_lost,attack_scored,defence_goals_conceeded,general_goal_difference,...,end_2012,end_2011,end_2010,Total,Champions League,Champions League Qualification,Europa League,Europa League Qualification,No UEFA Competitions,Relegated
0,1,26,150986355,38,32,2,4,95,23,72,...,-59.85,-146.46,-116.35,-1070.83,1,0,0,0,0,0
1,2,29,152425146,38,30,7,1,89,22,67,...,-43.63,3.98,5.05,-324.86,1,0,0,0,0,0
2,3,25,146030216,38,21,9,8,63,39,24,...,-64.05,-105.00,-26.20,-560.93,0,1,0,0,0,0
3,4,26,145230801,38,23,2,13,67,39,28,...,33.80,-23.67,-8.60,-16.78,0,1,0,0,0,0
4,5,25,142193180,38,21,7,10,73,51,22,...,12.82,-13.40,35.70,-281.72,0,0,1,0,0,0
5,6,27,142512868,38,19,9,10,65,54,11,...,-47.84,-11.99,77.07,-666.55,0,0,1,0,0,0
6,7,15,127165114,38,16,9,13,47,46,1,...,-9.45,-13.78,-18.58,-159.30,0,0,0,1,0,0
7,8,18,128603905,38,15,9,14,54,46,8,...,19.60,4.90,5.04,-208.19,0,0,0,0,1,0
8,9,15,123328078,38,15,7,16,51,48,3,...,-16.71,-2.50,-0.81,-168.69,0,0,0,0,1,0
9,10,16,122528663,38,15,7,16,52,55,-3,...,-2.49,-10.88,19.65,-221.72,0,0,0,0,1,0


## K-Means k=2

In [14]:
kmeans = KMeans(n_clusters=2, random_state=0).fit(result_encoding)

In [15]:
a = np.array(kmeans.labels_)
a.sum()

6

In [16]:
unique, counts = np.unique(a, return_counts=True)
dicionario = dict(zip(unique, counts))
print(dicionario)
lista = list(dicionario.items())
class_instances = pd.DataFrame(lista, columns=["Classe", "Nº Instancias"])
class_instances

{0: 13, 1: 6}


,Classe,Nº Instancias
0,0,13
1,1,6


### Silhouette

In [17]:
km_ = result_encoding.assign(cluster_number=kmeans.predict(result_encoding))
sk.metrics.silhouette_score(km_, labels=km_["cluster_number"])

0.7692706157307959

## K-Means k=5

In [18]:
kmeans = KMeans(n_clusters=5, random_state=0).fit(result_encoding)

In [19]:
a = np.array(kmeans.labels_)
a.sum()

37

In [20]:
unique, counts = np.unique(a, return_counts=True)
dicionario = dict(zip(unique, counts))
print(dicionario)
lista = list(dicionario.items())
class_instances = pd.DataFrame(lista, columns=["Classe", "Nº Instancias"])
class_instances

{0: 7, 1: 1, 2: 1, 3: 6, 4: 4}


,Classe,Nº Instancias
0,0,7
1,1,1
2,2,1
3,3,6
4,4,4


### Silhouette

In [21]:
km_ = result_encoding.assign(cluster_number=kmeans.predict(result_encoding))
sk.metrics.silhouette_score(km_, labels=km_["cluster_number"])

0.47013832606718325

## K-Means k=10

In [22]:
kmeans = KMeans(n_clusters=10, random_state=0).fit(result_encoding)

In [23]:
a = np.array(kmeans.labels_)
a.sum()

85

In [24]:
unique, counts = np.unique(a, return_counts=True)
dicionario = dict(zip(unique, counts))
print(dicionario)
lista = list(dicionario.items())
class_instances = pd.DataFrame(lista, columns=["Classe", "Nº Instancias"])
class_instances

{0: 4, 1: 1, 2: 1, 3: 1, 4: 1, 5: 2, 6: 2, 7: 4, 8: 2, 9: 1}


,Classe,Nº Instancias
0,0,4
1,1,1
2,2,1
3,3,1
4,4,1
5,5,2
6,6,2
7,7,4
8,8,2
9,9,1


### Silhouette

In [25]:
km_ = result_encoding.assign(cluster_number=kmeans.predict(result_encoding))
sk.metrics.silhouette_score(km_, labels=km_["cluster_number"])

0.46420915413668373

## K-Means k=19

In [26]:
kmeans = KMeans(n_clusters=19, random_state=0).fit(result_encoding)

In [27]:
a = np.array(kmeans.labels_)
a.sum()

171

In [28]:
unique, counts = np.unique(a, return_counts=True)
dicionario = dict(zip(unique, counts))
print(dicionario)
lista = list(dicionario.items())
class_instances = pd.DataFrame(lista, columns=["Classe", "Nº Instancias"])
class_instances

{0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1}


,Classe,Nº Instancias
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
5,5,1
6,6,1
7,7,1
8,8,1
9,9,1


In [29]:
# h_clustered = result_encoding.assign(cluster_number=clustering.fit_predict(result_encoding))
# sk.metrics.silhouette_score(h_clustered, labels=h_clustered["cluster_number"])

## Kmeans k=7 e max_iter=1

In [30]:
kmeans = KMeans(n_clusters=7, max_iter=1).fit(result_encoding)

km_clustered = result_encoding.assign(cluster_number=kmeans.predict(result_encoding))
#Silhouette
sk.metrics.silhouette_score(km_clustered, labels=km_clustered["cluster_number"])

0.4555277338959717

In [31]:
a = np.array(kmeans.labels_)
unique, counts = np.unique(a, return_counts=True)
dicionario = dict(zip(unique, counts))
print(dicionario)
lista = list(dicionario.items())
class_instances = pd.DataFrame(lista, columns=["Classe", "Nº Instancias"])
class_instances

{0: 6, 1: 2, 2: 3, 3: 1, 4: 1, 5: 2, 6: 4}


,Classe,Nº Instancias
0,0,6
1,1,2
2,2,3
3,3,1
4,4,1
5,5,2
6,6,4


## Kmeans k=7 e max_iter=10

In [32]:
kmeans = KMeans(n_clusters=7, max_iter=10).fit(result_encoding)

km_clustered = result_encoding.assign(cluster_number=kmeans.predict(result_encoding))
#Silhouette
sk.metrics.silhouette_score(km_clustered, labels=km_clustered["cluster_number"])

0.4555277338959705

In [33]:
a = np.array(kmeans.labels_)
unique, counts = np.unique(a, return_counts=True)
dicionario = dict(zip(unique, counts))
print(dicionario)
lista = list(dicionario.items())
class_instances = pd.DataFrame(lista, columns=["Classe", "Nº Instancias"])
class_instances

{0: 1, 1: 4, 2: 2, 3: 6, 4: 1, 5: 2, 6: 3}


,Classe,Nº Instancias
0,0,1
1,1,4
2,2,2
3,3,6
4,4,1
5,5,2
6,6,3


## Kmeans k=7 e max_iter=100

In [34]:
kmeans = KMeans(n_clusters=7, max_iter=100).fit(result_encoding)

km_clustered = result_encoding.assign(cluster_number=kmeans.predict(result_encoding))
#Silhouette
sk.metrics.silhouette_score(km_clustered, labels=km_clustered["cluster_number"])

0.45552773389597045

In [35]:
a = np.array(kmeans.labels_)
unique, counts = np.unique(a, return_counts=True)
dicionario = dict(zip(unique, counts))
print(dicionario)
lista = list(dicionario.items())
class_instances = pd.DataFrame(lista, columns=["Classe", "Nº Instancias"])
class_instances

{0: 3, 1: 2, 2: 6, 3: 2, 4: 1, 5: 4, 6: 1}


,Classe,Nº Instancias
0,0,3
1,1,2
2,2,6
3,3,2
4,4,1
5,5,4
6,6,1


## Hierarquico k=2 / linkage=ward

In [36]:
clustering = AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage="ward").fit(result_encoding)


In [37]:
a = np.array(clustering.labels_)
a

array([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [38]:
unique, counts = np.unique(a, return_counts=True)
dicionario = dict(zip(unique, counts))
print(dicionario)
lista = list(dicionario.items())
class_instances = pd.DataFrame(lista, columns=["Classe", "Nº Instancias"])
class_instances

{0: 6, 1: 13}


,Classe,Nº Instancias
0,0,6
1,1,13


### Silhouette

In [39]:
h_clustered = result_encoding.assign(cluster_number=clustering.fit_predict(result_encoding))
sk.metrics.silhouette_score(h_clustered, labels=h_clustered["cluster_number"])

0.7692706157307959

## Hierarquico k=2 / linkage=complete

In [40]:
clustering = AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage="complete").fit(result_encoding)


In [41]:
a = np.array(clustering.labels_)
a

array([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [42]:
unique, counts = np.unique(a, return_counts=True)
dicionario = dict(zip(unique, counts))
print(dicionario)
lista = list(dicionario.items())
class_instances = pd.DataFrame(lista, columns=["Classe", "Nº Instancias"])
class_instances

{0: 6, 1: 13}


,Classe,Nº Instancias
0,0,6
1,1,13


### Silhouette

In [43]:
h_clustered = result_encoding.assign(cluster_number=clustering.fit_predict(result_encoding))
sk.metrics.silhouette_score(h_clustered, labels=h_clustered["cluster_number"])

0.7692706157307959

## Hierarquico k=5 / linkage=complete

In [44]:
clustering = AgglomerativeClustering(n_clusters=5, affinity='euclidean', linkage="complete").fit(result_encoding)


In [45]:
a = np.array(clustering.labels_)
a

array([2, 1, 1, 1, 1, 3, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4, 4, 4, 4])

In [46]:
unique, counts = np.unique(a, return_counts=True)
dicionario = dict(zip(unique, counts))
print(dicionario)
lista = list(dicionario.items())
class_instances = pd.DataFrame(lista, columns=["Classe", "Nº Instancias"])
class_instances

{0: 6, 1: 4, 2: 1, 3: 1, 4: 7}


,Classe,Nº Instancias
0,0,6
1,1,4
2,2,1
3,3,1
4,4,7


### Silhouette

In [47]:
h_clustered = result_encoding.assign(cluster_number=clustering.fit_predict(result_encoding))
sk.metrics.silhouette_score(h_clustered, labels=h_clustered["cluster_number"])

0.47013832606718325

## Hierarquico k=5 / linkage=ward

In [48]:
clustering = AgglomerativeClustering(n_clusters=5, affinity='euclidean', linkage="ward").fit(result_encoding)


In [49]:
a = np.array(clustering.labels_)
a

array([2, 1, 1, 1, 1, 4, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [50]:
unique, counts = np.unique(a, return_counts=True)
dicionario = dict(zip(unique, counts))
print(dicionario)
lista = list(dicionario.items())
class_instances = pd.DataFrame(lista, columns=["Classe", "Nº Instancias"])
class_instances

{0: 10, 1: 4, 2: 1, 3: 3, 4: 1}


,Classe,Nº Instancias
0,0,10
1,1,4
2,2,1
3,3,3
4,4,1


### Silhouette

In [51]:
h_clustered = result_encoding.assign(cluster_number=clustering.fit_predict(result_encoding))
sk.metrics.silhouette_score(h_clustered, labels=h_clustered["cluster_number"])

0.5012077665339084

## Hierarquico k=19 / linkage=single

In [52]:
clustering = AgglomerativeClustering(n_clusters=19, affinity='euclidean', linkage="single").fit(result_encoding)


In [53]:
a = np.array(clustering.labels_)
a

array([11, 13, 17,  9, 15, 16, 10, 14,  4,  7, 18,  8, 12,  6,  3,  1,  5,
        2,  0])

In [54]:
unique, counts = np.unique(a, return_counts=True)
dicionario = dict(zip(unique, counts))
print(dicionario)
lista = list(dicionario.items())
class_instances = pd.DataFrame(lista, columns=["Classe", "Nº Instancias"])
class_instances

{0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1}


,Classe,Nº Instancias
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
5,5,1
6,6,1
7,7,1
8,8,1
9,9,1


### Silhouette

In [55]:
# h_clustered = result_encoding.assign(cluster_number=clustering.fit_predict(result_encoding))
# sk.metrics.silhouette_score(h_clustered, labels=h_clustered["cluster_number"])

## Considerando os dados acima

Podemos observar que os melhores resultados foram:

- Kmeans = 0.7692706157307959
- Hierarquico = 0.7692706157307959

E que os resultados com as mudanças no linkage tiveram seu pior resultado com single

As alterações em max_iter foram mais signficativas com k=5 ou k=7. Quando k=2 os resultados foram identicos.